## Length of Day

BB needs > 14.5 h daylength!!


### Presettings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import os
import re

import xarray as xr
#import cartopy.crs as ccrs

In [ ]:
# All directories
root       = "C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023"
this_dir   =  os.getcwd()

input_dir  = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Original" # to save space, not all data might be there anymore
output_dir = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Postprocessed"

### Methods

In [ ]:
# Source: https://gist.github.com/anttilipp/ed3ab35258c7636d87de6499475301ce

def daylength(dayOfYear, lat):
    """Computes the length of the day (the time between sunrise and
    sunset) given the day of the year and latitude of the location.
    Function uses the Brock model for the computations.
    For more information see, for example,
    Forsythe et al., "A model comparison for daylength as a
    function of latitude and day of year", Ecological Modelling,
    1995.

    Parameters
    ----------
    dayOfYear : int
        The day of the year. 1 corresponds to 1st of January
        and 365 to 31st December (on a non-leap year).
    lat : float
        Latitude of the location in degrees. Positive values
        for north and negative for south.

    Returns
    -------
    d : float
        Daylength in hours.
    """
    
    latInRad = np.deg2rad(lat)
    
    declinationOfEarth = 23.45*np.sin(np.deg2rad(360.0*(283.0+dayOfYear)/365.0))
    
    if -np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth)) <= -1.0:
        return 24.0
    
    elif -np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth)) >= 1.0:
        return 0.0
    
    else:
        hourAngle = np.rad2deg(np.arccos(-np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth))))
        return 2.0*hourAngle/15.0


# Execution

In [ ]:
# use a dummy dataset to get latitudes of the copernicus data
dummy = xr.open_dataset(f"{obs_path}/tempavg_dailymean_1980-1994_0.25deg.nc")
dummy.close()
dummylat = maxgen.latitude.values
del dummy

In [ ]:
# generate times for when to check length of day (beginning and end of each month) according to latitude
timestamp = []
dayOfYear = []

for y in range(1980,2023):
    for m in range (1,13):
        for d in range(1, pd.Timestamp(y, m, 1).daysinmonth):
        
            timestamp.append(pd.Timestamp(y, m, d))
            dayOfYear.append(pd.Timestamp(y, m, d).dayofyear)
                    
# check all these times for every lat

In [ ]:
# generate data
dat = []
for lat in dummylat:
    
        subdat = [daylength(DoY, lat) for DoY in dayOfYear]
        dat.append(subdat)
        
dat = np.array(dat)

In [ ]:
# generate dataarray
da = xr.DataArray(

    data  = dat,

    dims  = ["latitude", "time"],

    coords= dict(
        latitude = dummylat,
        time     = timestamp,
    )
)

da = da.to_dataset(name='daylength')
da = da.assign_attrs(dict(description="length of day for middle of each month depending on latitude",
                     units="hours"))

del dat
da
#da.daylength.isel(latitude=100).isel(time=slice(0,11)).plot()

In [ ]:
# Export
da.to_netcdf(f"{obs_path}/daylight_daily_europe_1980-2022_0.25deg.nc")

In [ ]:
# Sammple Lat & long von Bezirksmitte
# Zuerst muss ich wissen, WO ist der Fichtenwald?